# SVM

# Kernel function

A kernel function is a real-valued function of two arguments, $k(\mathbf{x}, \mathbf{x}') \in \mathbb{R}$ for $ \mathbf{x}, \mathbf{x}' \in \mathcal{X} $, where $ \mathcal{X}$ is the input space.